In [ ]:
import tensorflow as tf
import numpy as np

import sys, os; sys.path.append(os.path.expanduser('~/code/tf-gqn'))

from notebooks.jupyter_extras import show_graph

from data_provider.gqn_tfr_provider import input_fn
from gqn.gqn_graph import gqn
from gqn.gqn_params import PARAMS
from gqn.gqn_utils import eta_g

In [ ]:
DATA_ROOT='/Volumes/Tiberius/gqn-dataset/'
BAR_WIDTH=2

In [ ]:
tf.reset_default_graph()

data = input_fn(dataset='shepard_metzler_5_parts',
    context_size=5,
    root=DATA_ROOT,
    mode=tf.estimator.ModeKeys.TRAIN,
    batch_size=1,
    num_epochs=None)

net, endpoints = gqn(
    query_pose=data.query.query_camera, 
    target_frame=data.target,
    context_poses=data.query.context.cameras, 
    context_frames=data.query.context.frames,
    model_params=PARAMS)

def debug_canvas_image(canvases, eta_g_scope='GQN'):
    with tf.variable_scope(eta_g_scope, reuse=True, auxiliary_name_scope=False), \
         tf.name_scope('debug'):
        mean_images = []

        with tf.name_scope('MakeWhiteBar'):
            cs = tf.shape(canvases[0])
            batch, height, channels = cs[0], cs[1], 3
            white_vertical_bar = tf.ones(shape=(batch, height, BAR_WIDTH, channels), 
                                         dtype=tf.float32,
                                         name='white_bar')

        for canvas in canvases:   
            mean_images.append(eta_g(canvas))
            mean_images.append(white_vertical_bar)

        return tf.concat(mean_images[:-1], axis=-2, name='MakeGrid')

image_sequence = debug_canvas_image([endpoints['canvas_{}'.format(i)] for i in range(12)])

show_graph(tf.get_default_graph())

In [ ]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    im = s.run(image_sequence)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(32,32))
plt.imshow(im[0,:,:,:])